 # 31. ML 모델링 with feature engineering

 변수를 추가해 봅시다.

# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

import scipy.stats as spst
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

## (2) 함수 생성

### 1) 결과 시각화

In [ ]:
def plot_model_result(y_train, y_val, pred) :
    pred = pd.Series(pred, index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

### 2) 잔차분석

In [ ]:
def residual_diag(residuals, lags = 30) :
    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

### 3) 시계열분해 plot

In [ ]:
def decomp_plot(decomp) :
    result = pd.DataFrame({'observed':decomp.observed, 'trend':decomp.trend, 'seasonal':decomp.seasonal, 'residual':decomp.resid})
    plt.subplot(4,1,1)
    plt.plot(result['observed'])
    plt.ylabel('observed')
    plt.subplot(4,1,2)
    plt.plot(result['trend'])
    plt.ylabel('trend')
    plt.subplot(4,1,3)
    plt.plot(result['seasonal'])
    plt.ylabel('seasonal')
    plt.subplot(4,1,4)
    plt.plot(result['residual'])
    plt.ylabel('residual')
    plt.show()

    return result

## (3) 데이터 불러오기

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/SeoulBike_Simple.csv'
bike = pd.read_csv(path)
bike['Datetime'] = pd.to_datetime(bike['Datetime'] )
bike.rename(columns={'Rented Bike Count':'Count'}, inplace = True)
bike = bike.loc[bike['Datetime'].between('2018-06-11','2018-08-13', inclusive = 'left'),
                      ['Datetime', 'Temperature', 'Humidity','Count']]
bike.reset_index(drop = True, inplace = True)

In [ ]:
bike

## (4) 데이터 둘러보기

In [ ]:
bike.describe(include = 'all').T

In [ ]:
# 마지막 14일의 그래프를 그려 봅시다.
size = 24 * 14
temp = bike.iloc[-size:]
plt.figure(figsize = (20,8))
plt.plot('Datetime', 'Count', data = temp)
plt.grid()
plt.show()

# 2.전처리

## (1) y 만들기

In [ ]:
data = bike.copy()

In [ ]:
data['y'] = data['Count'].shift(-2)

In [ ]:
data = data.iloc[:-2]

## (2) NaN 조치

In [ ]:
data.fillna(method = 'ffill', inplace = True)

## (3) Feature Engineering

### 1) 날짜 요소 뽑기

| 메서드 | 내용|
|----|----|
df['date'].dt.date         		| YYYY-MM-DD(문자)
df['date'].dt.year         		| 연(4자리숫자)
df['date'].dt.month        		| 월(숫자)
df['date'].dt.month_name()		| 월(문자)
df['date'].dt.day          		| 일(숫자)
df['date'].dt.time         		| HH:MM:SS(문자)
df['date'].dt.hour         		| 시(숫자)
df['date'].dt.minute       		| 분(숫자)
df['date'].dt.second       		| 초(숫자)
df['date'].dt.quarter       		| 분기(숫자)
df['date'].dt.day_name()  	| 요일이름(문자)
df['date'].dt.weekday       		| 요일숫자(0-월, 1-화) (=dayofweek)
df['date'].dt.weekofyear    		| 연 기준 몇주째(숫자) (=week)
df['date'].dt.dayofyear     		| 연 기준 몇일째(숫자)
df['date'].dt.days_in_month 	| 월 일수(숫자) (=daysinmonth)



#### ① 요일 --> 주말여부
* 먼저 요일을 뽑고,
* 주말 여부로 변환

In [ ]:
data['Datetime'].dt.weekday

In [ ]:
np.where(data['Datetime'].dt.weekday < 5, 0, 1)

In [ ]:
data['weekend'] = np.where(data['Datetime'].dt.weekday < 5, 0, 1)
data.head()

#### ② 활동시간 구분
* 시간을 추출하고, 다음과 같이 범주로 생성합니다.
    * 0 : 활동 없음, 1 : 일상 활동시간, 2 : 출퇴근
    * 0~6 : 0
    * 7~8 : 2
    * 9~16: 1
    * 17~20 : 2
    * 21~23 : 0


In [ ]:
data['Active'] = pd.cut(data['Datetime'].dt.hour, bins=[0,6,8,16,20,23], labels = [0,2,1,2,0],
                        ordered = False, include_lowest = True)

### 2) 시간의 흐름을 feature로 담기

#### ① shift

In [ ]:
temp = data[['Datetime', 'y']].copy()
temp.head()

In [ ]:
# shift(1)
temp['lag1'] = temp['y'].shift()

# shift(2)
temp['lag2'] = temp['y'].shift(2)

# shift(-1)
temp['lag_1'] = temp['y'].shift(-1)
temp.head()

* 24시간 전 수요량

In [ ]:
data['lag24'] = data['y'].shift(24)
data.head(30)

#### ② rolling

In [ ]:
temp = data[['Datetime', 'y']].copy()
temp.head()

In [ ]:
# 3일 이동평균
temp['MA3'] = temp['y'].rolling(3).mean()

# 3일 이동최대값
temp['MM3'] = temp['y'].rolling(3).max()

# 3일 이동평균(min_period = 1)
temp['MA3_1'] = temp['y'].rolling(3, min_periods=1).mean()

temp.head()

* 4시간 이동평균

In [ ]:
data['MA4'] = data['Count'].rolling(4, min_periods = 1).mean()
data.head(10)

#### ③ diff
특정 시점 대비 증감

In [ ]:
temp = data[['Datetime', 'y']].copy()
temp.head()

In [ ]:
# 전일대비 증감
temp['Diff1'] = temp['y'].diff()

# 2일 전 대비 증가
temp['Diff2'] = temp['y'].diff(2)

temp.head()

* 전 시간 대비 증감

In [ ]:
data['Diff1'] = data['Count'].diff()

* 24시간 전 대비 증감

In [ ]:
data['Diff24'] = data['Count'].diff(24)

data.head(30)

### 3) decompose

* 시계열 데이터 분해는,
    * 시계열 데이터 안에 있는 반복 추세와 반복 패턴을 찾아내는 과정입니다.
    * 찾아낸 패턴을 하나의 모델로 볼 수도 있습니다.
    * 여기서는 과거의 패턴을 하나의 feature로 도출해 보겠습니다.

#### ① 시계열 데이터를 분해해 봅시다.
* freq 를 조절하며 계절성(seasonal)을 도출해 봅시다.
* 그래프는 위에서 만든 decomp_plot 을 사용합니다.

* 1일(24시간)의 계절성 고려

In [ ]:
decomp = sm.tsa.seasonal_decompose(data['y'], model = 'additive', period = 24)
plt.figure(figsize=(12,10))
result = decomp_plot(decomp)

* 1주일(7일 * 24시간)의 계절성 고려

In [ ]:
decomp = sm.tsa.seasonal_decompose(data['y'], model = 'additive', period = 7*24)
plt.figure(figsize=(12,10))
result = decomp_plot(decomp)

In [ ]:
result.head()

#### ② seasonal 데이터를 New Feature로 추가합시다.
* 단, train 데이터만을 이용해서 생성한 후 전체에 입력해야 합니다.
* val size : 24 * 14 이므로 이 데이터를 빼고 나머지만 가지고 시계열 데이터 분할

* seasonal 패턴1 저장
    * period = 24 이므로, seasonal 패턴은 24시간 주기

In [ ]:
val_size = 24 * 14
decomp = sm.tsa.seasonal_decompose(data.y[:-val_size], model = 'additive', period = 24)
plt.figure(figsize=(12,10))
result = decomp_plot(decomp)

In [ ]:
display(data.head(3))
display(data.iloc[24:27])

In [ ]:
display(result.head(3))
display(result.iloc[24:27])

In [ ]:
data.iloc[-val_size:]

In [ ]:
# 24개 주기 seasonal 데이터 붙이기
seasonal24 = result.seasonal[:24]

# 전체 데이터 건수 만큼 Seasonal 데이터 만들기
n = data.shape[0] // 24 + 1
seasonal_all = list(seasonal24) * n
seasonal_all[: data.shape[0]]

# data 셋에 붙이기
data['Seasonal24'] = seasonal_all[: data.shape[0]]
data.head(10)

* seasonal 패턴2 저장
    * period = 7 * 24

In [ ]:
val_size = 24 * 14
decomp = sm.tsa.seasonal_decompose(data.y[:-val_size], model = 'additive', period = 24*7)
plt.figure(figsize=(12,10))
result = decomp_plot(decomp)

In [ ]:
# 24 * 7개 주기 seasonal 데이터 붙이기
seasonal168 = result.seasonal[:24 * 7]

# 전체 데이터 건수 만큼 Seasonal 데이터 만들기
n = data.shape[0] // 168 + 1
seasonal_all = list(seasonal168) * n
seasonal_all[: data.shape[0]]

# data 셋에 붙이기
data['Seasonal168'] = seasonal_all[: data.shape[0]]
data.head(10)

## (4) NaN 추가 조치

In [ ]:
data.isna().sum()

* 여기서 NaN은 삭제합시다.

In [ ]:
data1 = data.dropna(axis = 0)
data1.reset_index(drop = True, inplace = True)

## (5) 가변수화

In [ ]:
# 요일 변수를 가변수화
data2 = pd.get_dummies(data1, columns = ['Active'], drop_first = True )
data2.head()

## (6) 데이터 분할

### 1) x, y 나누기

* .values(넘파이 어레이)로 변환해서 저장하는 이유 ➡ 데이터 스플릿 index를 적용해서 데이터를 가져오기 위해서

In [ ]:
target = 'y'

x = data2.drop([target, 'Datetime'], axis = 1)
y = data2.loc[:, target]

### 2) 시계열 데이터 분할

In [ ]:
val_size = 24 * 14
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = val_size, shuffle = False)

# 3.모델링1 : Linear Regression

## (1) 학습 및 예측

### 1) 학습

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model1 = LinearRegression()
model1.fit(x_train, y_train)

### 2) 예측

In [ ]:
# 예측
pred = model1.predict(x_val)

## (3) 평가

### 1) 검증성능

In [ ]:
# 평가
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  :', r2_score(y_val, pred))

### 2) 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

# 4.모델링2 : XGB

## (1) 학습 및 예측

### 1) 학습

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# 1~2분 소요
# hyper parameter 설정
params = {'n_estimators':range(10, 101, 10), 'learning_rate':np.linspace(0.001, 0.3, 20)}

# Grid Search
model2 = GridSearchCV(XGBRegressor(), params, cv = 3, verbose = 2 )
model2.fit(x_train, y_train)

In [ ]:
model2.best_params_

### 2) 예측

In [ ]:
# 예측
pred = model2.predict(x_val)

# 잔차 : 실제값에서 예측값을 빼서 계산
residuals = y_val - pred

## (3) 평가

### 1) 검증성능

In [ ]:
# 평가
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  :', r2_score(y_val, pred))

### 2) 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

# 5.전처리 데이터 저장
* joblib을 이용하여 데이터프레임 저장하기

In [ ]:
import joblib

joblib.dump(data2, 'data2.pkl')